In [5]:
def get_asciiz(barray, pos):
    str=''
    while barray[pos] != 0:
        str += chr(barray[pos])
        pos += 1
    return str

def get_dword(barray, pos):
    val=0
    for i in range(3, -1, -1):
        val = val<<8 | barray[pos+i]
    return val

def get_word(barray, pos):
    val=0
    for i in range(1, -1, -1):
        val = val<<8 | barray[pos+i]
    return val

def get_header(img):
    image_name = get_asciiz(img, 0)
    write_protect = img[0x1a]
    disk_type = img[0x1b]
    disk_size = get_dword(img, 0x1c)
    hdr= { 'name'          : image_name, 
           'write_protect' : 'ON' if write_protect==0x10 else 'OFF',
           'disk_type'     : '2D' if disk_type==0 else '2DD' if disk_type==0x10 else '2HD',
           'disk_size'     : '0x'+format(disk_size, '08X')
         }
    return hdr

def get_sect(img, ofst):
    CHRN = ( img[ofst + 0], img[ofst + 1], img[ofst + 2], img[ofst +3 ])
    num_sec = get_word(img, ofst+4)
    density = img[ofst+6]
    address_mark = img[ofst+7]
    status = img[ofst+8]
    size = get_word(img, ofst+0x0e)
    data = img[ofst:ofst+size+1]
    return { 'CHRN' : '{}:{}:{}:{}'.format(CHRN[0], CHRN[1], CHRN[2], CHRN[3]),
              'num_sec' : num_sec,
              'density' : 'Double density' if density==0 else 'Single density',
              'status' : status,
              'am' : address_mark,
              'size' : size,
              'data' : data }

def decode_d77(file):
    with open(file, 'rb') as f:
        img = f.read()

    d77 = {}

    # Extract Header information
    d77['header'] = get_header(img)

    # Extract track offset
    trk_ofst = [0]*164
    for t in range(164):
        trk_ofst[t] = get_dword(img, 0x20 + t*4)
        #print('{} {:08X}'.format(t, trk_ofst[t]))
    d77['trk_ofst'] = trk_ofst

    # Extract sector data
    disk = []
    for t in range(164):
        disk.append([])
        ofst = trk_ofst[t]
        if ofst == 0:
            continue
        sect = get_sect(img, ofst)
        num_sect = sect['num_sec']
        for i in range(num_sect):
            sect = get_sect(img, ofst)
            disk[-1].append(sect)
            ofst += 0x10 + sect['size']
    d77['disk'] = disk
    return d77

In [10]:
#d77 = decode_d77('putty/cdos7-4.d77')
d77 = decode_d77('putty/thexder.d77')

# Display results
print('Header:', d77['header'])

disk = d77['disk']
for trkid, track in enumerate(disk):
    print(trkid)
    for sect in track:
        c,h,r,n = sect['CHRN'].split(':')
        status = sect['status']
        size = sect['size']
        am = sect['am']
        print('{:02x} {:02x} {:02x} {:02x} 0x{:02x} {:02x} {:4}'.format(int(c),int(h),int(r),int(n), status, am, size))

Header: {'name': 'DISK', 'write_protect': 'OFF', 'disk_type': '2D', 'disk_size': '0x000D1103'}
0
00 00 01 01 0x00 00  256
00 00 02 01 0x00 00  256
00 00 03 01 0x00 00  256
00 00 04 01 0x00 00  256
00 00 05 01 0x00 00  256
00 00 06 01 0x00 00  256
00 00 07 01 0x00 00  256
00 00 08 01 0x00 00  256
00 00 09 01 0x00 00  256
00 00 0a 01 0x00 00  256
00 00 0b 01 0x00 00  256
00 00 0c 01 0x00 00  256
00 00 0d 01 0x00 00  256
00 00 0e 01 0x00 00  256
00 00 0f 01 0x00 00  256
00 00 10 01 0x00 00  256
1
00 01 01 01 0x00 00  256
00 01 02 01 0x00 00  256
00 01 03 01 0x00 00  256
00 01 04 01 0x00 00  256
00 01 05 01 0x00 00  256
00 01 06 01 0x00 00  256
00 01 07 01 0x00 00  256
00 01 08 01 0x00 00  256
00 01 09 01 0x00 00  256
00 01 0a 01 0x00 00  256
00 01 0b 01 0x00 00  256
00 01 0c 01 0x00 00  256
00 01 0d 01 0x00 00  256
00 01 0e 01 0x00 00  256
00 01 0f 01 0x00 00  256
00 01 10 01 0x00 00  256
2
01 00 01 01 0x00 00  256
01 00 02 01 0x00 00  256
01 00 03 01 0x00 00  256
01 00 04 01 0x00 00  256

In [ ]:
}